In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import json

import evaluate
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    AutoModelForCausalLM
)
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Seed Set
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# 디바이스 설정 (GPU가 사용 가능하면 GPU를 사용하고, 그렇지 않으면 CPU 사용)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
data_path = os.path.join('..', '..', 'data',"v0.1","train.csv")
data = pd.read_csv(data_path)
data = data.drop(columns=["Unnamed: 0"])

In [6]:
data

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,3
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,1
3,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
...,...,...,...
1499,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1500,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1501,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4
1502,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [13]:
label0_data = data[data["target"]==0]
label1_data = data[data["target"]==1]
label2_data = data[data["target"]==2]
label3_data = data[data["target"]==3]
label4_data = data[data["target"]==4]
label5_data = data[data["target"]==5]
label6_data = data[data["target"]==6]

In [21]:
def get_few_shot_data(label_data, num):
    few_shot_data = label_data.sample(num,random_state=456)
    few_shot_list = [{"input":few_shot_data.iloc[i]["text"]} for i in range(len(few_shot_data))]
    return few_shot_list

In [7]:
model_id = "rtzr/ko-gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [35]:
few_shot_list_0

[{'input': '서울에 다시 오존주의보…도심·서북·동북권 발령종합'},
 {'input': '경기도 11개 시 오존주의보 해제'},
 {'input': '북한날씨 아침에 비 대부분 그쳐'},
 {'input': '우산 챙기세요…전국 구름 많고 소나기'},
 {'input': '대구·경북 수은주 뚝…내일 새벽 산지엔 서리·얼음'},
 {'input': '먼지공습 걷혔다…경기 전역 미세먼지주의보 해제'},
 {'input': '작년 우리나라 낙뢰 7월에만 18만번…벼락 맞을 확률은'},
 {'input': '폭염엔 역시 얼음'},
 {'input': '충북 말복 더위 지속…강한 소나기 주의'},
 {'input': '전국 아침 기온 최대 7도 뚝…주말까지 쌀쌀'},
 {'input': '날씨-미세먼지 보통 수준…오후부터 지속적인 고온, 곳곳 폭염 예상  '},
 {'input': '휴일, 청객 울산{미세먼지} 주의 발령 '},
 {'input': '추워져 눈 내리고, 기온 하락  '},
 {'input': '밤낮없는 무더위…강릉 30.1도 등 강원 대부분 열대야'},
 {'input': '제주 낮최고 22.5도 포근…산간 호우 예비특보'},
 {'input': '8월 중순 중부에 물폭탄…평균 강수량 223.4㎜ 평년 2배'},
 {'input': '내주날씨 화요일까지 전국에 비…주 후반 찜통더위'},
 {'input': '더위 식히는 장맛비…남부·제주도 밤에 대부분 그쳐'},
 {'input': '백두대간의 가을 하늘'},
 {'input': '날씨 6일 전국 흐리고 비…태풍 영향권 제주·남해안 강풍'}]

In [70]:
def prompt_few_shot(few_shot_list):
    system_message = """당신은 뉴스 기사 제목을 생성하는 어시스턴트입니다. 다음과 같은 지침을 따르세요.

    1. 주어진 예시와 같은 주제, 다른방식으로 새로운 뉴스 기사 제목을 생성하세요.
    2. 한줄 내외로 생성하세요.
    3. 이모티콘은 포함하지 마세요.
    4. 주어진 예시와 비슷한 내용은 생성하지 마세요.
    5. 제목의 톤은 중립적이며 뉴스 형식에 맞춰주세요.
    
    다음은 몇 가지 뉴스 기사 제목 예시입니다:

    """
    few_shot_messages = []
    for few_shot_data in few_shot_list:
        few_shot_messages.append(
            {"role":"user","content":"뉴스 기사 제목을 생성해주세요"}
        )
        few_shot_messages.append(
            {"role":"assistant","content":few_shot_data["input"]}
        )
    messages = [
        [{"role":"system", "content":system_message}] +
        few_shot_messages +
        [{"role":"user", "content":"뉴스 기사 제목을 생성해주세요"}]
    ]
    return messages

In [ ]:
few_shot_list_0 = get_few_shot_data(label0_data, 40)
few_shot_list_1 = get_few_shot_data(label1_data, 40)
few_shot_list_2 = get_few_shot_data(label2_data, 40)
few_shot_list_3 = get_few_shot_data(label3_data, 40)
few_shot_list_4 = get_few_shot_data(label4_data, 40)
few_shot_list_5 = get_few_shot_data(label5_data, 40)
few_shot_list_6 = get_few_shot_data(label6_data, 40)

def generate_sentence(messages, few_shot_list):
    messages = prompt_few_shot(few_shot_list)
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.9,
        top_p=0.92,
    )
    decoded_data = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    generate_data.append(decoded_data)
    decoded_data = decoded_data.replace("\n","")
    # few_shot_list.append({"input":decoded_data})
    print("GEN :",decoded_data)
    return decoded_data

generated_datas = []
lists = [few_shot_list_0,few_shot_list_1,few_shot_list_2,few_shot_list_3,few_shot_list_4,few_shot_list_5,few_shot_list_6]

for i, few_shots in enumerate(lists):
    for _ in range(100):
        gen_data = generate_sentence(messages, few_shots)
        generated_datas.append([gen_data, i])

GEN : 태풍 영향 남부·제주도 강수 폭발…일부 지역 최대 300mm  강수량
GEN : 일주일간 장마 지속…특히 강원도 60mm 이상 강수 예상
GEN : 강원도 속초·홍천·철원 등 8개 지역 폭우·landslide경고  
GEN : 강수량 172㎜…부산·울산 폭우 대피  주민 피해 경고
GEN : 강원도, 폭풍 해안 파도…저녁부터 강수확률 증가
GEN : 폭염 대비…체온 낮추는 똑똑한 쿨링팁 공개
GEN : 내일 날씨 습도 높고 쾌적…비확률 적어
GEN : 8월 들어 강수량 평년 대비 100mm 적은 '여름 고온, 안개 겹쳐'  
GEN : 내일부터 폭염 대유…서울 35도까지  날씨
GEN : 태풍 7급으로 강화…주말 강풍·집중호우 대비
GEN : 경기도·강원도 미세먼지주의보 발령…일시적으로 농작물 피해 우려
GEN : 전국 대부분 지역 맑고 습도 낮아…폭염 이슈는 잠시 멈춤  
GEN : 6일부터 비 오리…강원도 전역 구름 많고 우박 예상
GEN : 남부 한낮 폭염 넘어…영남 밤늦게까지 찜통더위
GEN : 내일날씨 흐림…비 오면서 기온 한참 하락
GEN : 전남 해안선 강한 파도…바다 위험상태 경고 
GEN : 강원도 내륙 지역 대기오염  
GEN : 2일 오전부터 기온 급등…내일은 28도까지   높아  
GEN : 강원도 습도 90% 넘어… '찜찜함'에 시민 불편증 증가
GEN : 8월까지 폭염 걱정…평균 기온 27.8도, 20년 만의 극심함
GEN : 전북 서해안 주말에 물폭탄 예상…경주·안동 폭염 주의
GEN : 장마 첫 폭우 쏟아져…지역 내수·피해 발생
GEN : 국가 기상청, 태풍 "나리" 3등급으로 발달 예상
GEN : 내일날씨 낮 기온 30도 돌파…중부지방은 32도까지 맹폭염
GEN : 전국 맑고 기온 상승…주말엔 30도 넘을 지역 생길 것 
GEN : 강원도, 이번 주 맑은 날씨 만끽…단풍 절정 시즌  접어들고
GEN : 강원도, 오후 1시 이후 비 시작…전북·전남까지 호우 예상
GEN : 30도 돌파 서울…내일부터 밤낮으로 찜통 놓치지 

In [90]:
gen_data = pd.DataFrame()
gen_texts = []
gen_targets = []

for gen_datas in generated_datas:
    gen_text, gen_target = gen_datas
    gen_texts.append(gen_text)
    gen_targets.append(gen_target)


In [91]:
aug_prefix = "aug-v1_gem_train_"
aug_prefix_id = []
for i in range(len(gen_texts)):
    aug_id = aug_prefix+str(i)
    aug_prefix_id.append(aug_id)


In [92]:
gen_data["ID"] = aug_prefix_id
gen_data["text"] = gen_texts
gen_data["target"] = gen_targets

In [96]:
gen_data = gen_data.sample(frac=1).reset_index(drop=True)

In [97]:
gen_data

,ID,text,target
0,aug-v1_gem_train_640,美 중국 탈출시킬까…이란 잠재적 협상 파트너로 급부상,6
1,aug-v1_gem_train_595,지속적인 금리 인상에 신용카드 대출 급증 초점,5
2,aug-v1_gem_train_530,"세금 납부 지연, 개인소득공제 혜택도 제한",5
3,aug-v1_gem_train_562,LG엔솔 정규직 채용 10년 만에 25% 감소,5
4,aug-v1_gem_train_401,"KT, 월드컵 특선 통신 상품 출시…해외 팬 위한 최적의 통신 환경 제공",4
...,...,...,...
695,aug-v1_gem_train_201,"이원재 의원 측 ""사재발 기업 몰락이 당의 부활의 기회""",2
696,aug-v1_gem_train_176,2023 프로야구 드래프트 한국시리즈 우승팀 3명 지명,1
697,aug-v1_gem_train_15,전남 해안선 강한 파도…바다 위험상태 경고,0
698,aug-v1_gem_train_579,"KCC, 투자 유치 성공…새로운 성장 동력 확보",5


In [98]:
gen_data.to_csv(os.path.join('..', '..', 'data',"v0.2","gemma_gen.csv"),encoding="utf-8-sig")